In [0]:
%python
spark.conf.set("spark.databricks.io.cache.enabled",False)

Loading Data

In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/flight_data_z",True)

True

In [0]:
%sql
DROP TABLE IF EXISTS flight_data_z;
CREATE TABLE flight_data_z(
  Year INT,Month INT,DayofMonth INT,DayofWeek INT,DepTime STRING, CRSDepTime INT,ArrTime STRING,CRSArrTime INT,UniqueCarrier STRING, FlightNum INT, TailNum STRING, ActualElapsedTime STRING,CRSElapsedTime STRING,AirTime STRING,
  ArrDelay STRING, DepDelay STRING, Origin STRING,Dest STRING,Distance INT, TaxiIn STRING,TaxiOut STRING,Cancelled INT, CancellationCode STRING, Diverted INT,CarrierDelay STRING, WeatherDelay STRING, NASDelay STRING, SecurityDelay STRING,LateAircraftDelay STRING);

In [0]:
%python
spark.read.csv("dbfs:/databricks-datasets/asa/airlines/2*.csv",header=True,inferSchema=True)\
    .write.mode("append").saveAsTable("flight_data_z")

Z order by Year and Origin

In [0]:
%sql
OPTIMIZE flight_data_z
ZORDER BY (Year,Origin) 
-- takes times , its a expensive operation

path,metrics
dbfs:/user/hive/warehouse/flight_data_z,"List(3, 47, List(309484081, 322596443, 3.17805019E8, 3, 953415057), List(213796, 26211765, 2.000750576595745E7, 47, 940352771), 0, List(minCubeSize(107374182400), List(0, 0), List(47, 940352771), 0, List(47, 940352771), 1, null), null, 0, 1, 47, 0, false, 0, 0, 1738829232229, 1738830046716, 8, 1, null, List(0, 0), 29, 29, 1399463, 0, null)"


In [0]:
%sql
DESC DETAIL flight_data_z

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,c86147f8-a8df-48b7-9502-99a385cd7934,spark_catalog.default.flight_data_z,null,dbfs:/user/hive/warehouse/flight_data_z,2025-02-06T07:49:47.279Z,2025-02-06T08:20:44Z,List(),List(),3,953415057,Map(),1,2,"List(appendOnly, invariants)",Map()


In [0]:
%sql
DESCRIBE HISTORY flight_data_z

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-02-06T08:20:44Z,5755764547042441,inaya998877@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> false, clusterBy -> [], zOrderBy -> [""Year"",""Origin""], batchId -> 0)",null,List(4081512079520275),0206-074511-wna9kc41,1,SnapshotIsolation,false,"Map(numRemovedFiles -> 47, numRemovedBytes -> 940352771, p25FileSize -> 309484081, numDeletionVectorsRemoved -> 0, minFileSize -> 309484081, numAddedFiles -> 3, maxFileSize -> 322596443, p75FileSize -> 322596443, p50FileSize -> 321334533, numAddedBytes -> 953415057)",null,Databricks-Runtime/15.4.x-scala2.12
1,2025-02-06T08:07:07Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4081512079520275),0206-074511-wna9kc41,0,WriteSerializable,true,"Map(numFiles -> 47, numOutputRows -> 59285457, numOutputBytes -> 940352771)",null,Databricks-Runtime/15.4.x-scala2.12
0,2025-02-06T07:49:49Z,5755764547042441,inaya998877@gmail.com,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {}, statsOnLoad -> false)",null,List(4081512079520275),0206-074511-wna9kc41,null,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-scala2.12


In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled=false;
VACUUM flight_data_z RETAIN 0 HOURS;

path
dbfs:/user/hive/warehouse/flight_data_z


In [0]:
%fs ls dbfs:/user/hive/warehouse/flight_data_z

path,name,size,modificationTime
dbfs:/user/hive/warehouse/flight_data_z/_delta_log/,_delta_log/,0,0
dbfs:/user/hive/warehouse/flight_data_z/part-00000-eaba37fb-967c-4a82-928b-8f86364eb0ff-c000.snappy.parquet,part-00000-eaba37fb-967c-4a82-928b-8f86364eb0ff-c000.snappy.parquet,322596443,1738830029000
dbfs:/user/hive/warehouse/flight_data_z/part-00001-3456dff5-858f-462c-af0d-d8a35ee04c43-c000.snappy.parquet,part-00001-3456dff5-858f-462c-af0d-d8a35ee04c43-c000.snappy.parquet,321334533,1738830041000
dbfs:/user/hive/warehouse/flight_data_z/part-00002-d9fb0979-78b0-4430-8c36-12f91e886e36-c000.snappy.parquet,part-00002-d9fb0979-78b0-4430-8c36-12f91e886e36-c000.snappy.parquet,309484081,1738830036000


In [0]:
%fs ls dbfs:/user/hive/warehouse/flight_data_z/_delta_log

path,name,size,modificationTime
dbfs:/user/hive/warehouse/flight_data_z/_delta_log/00000000000000000000.crc,00000000000000000000.crc,4136,1738828207000
dbfs:/user/hive/warehouse/flight_data_z/_delta_log/00000000000000000000.json,00000000000000000000.json,3159,1738828189000
dbfs:/user/hive/warehouse/flight_data_z/_delta_log/00000000000000000001.crc,00000000000000000001.crc,100646,1738829231000
dbfs:/user/hive/warehouse/flight_data_z/_delta_log/00000000000000000001.json,00000000000000000001.json,97391,1738829227000
dbfs:/user/hive/warehouse/flight_data_z/_delta_log/00000000000000000002.crc,00000000000000000002.crc,10725,1738830047000
dbfs:/user/hive/warehouse/flight_data_z/_delta_log/00000000000000000002.json,00000000000000000002.json,26351,1738830044000
dbfs:/user/hive/warehouse/flight_data_z/_delta_log/_commits/,_commits/,0,0


In [0]:
%sql
SELECT * FROM JSON.`dbfs:/user/hive/warehouse/flight_data_z/_delta_log/00000000000000000002.json`

add,commitInfo,remove
null,"List(0206-074511-wna9kc41, Databricks-Runtime/15.4.x-scala2.12, false, SnapshotIsolation, List(4081512079520275), OPTIMIZE, List(322596443, 309484081, 953415057, 3, 0, 940352771, 47, 309484081, 321334533, 322596443), List(false, 0, [], [], [""Year"",""Origin""]), 1, List(false), 1738830043157, 57837a43-e2d6-4922-a40e-467125e85c5a, 5755764547042441, inaya998877@gmail.com)",null
null,null,"List(false, 1738829236392, true, part-00000-0ca1daff-eb28-4d7e-9bd3-a360407adbee-c000.snappy.parquet, 16372673, {""numRecords"":1341343}, List(1738828509000000, 1738828509000000, 1738828509000000, 268435456))"
null,null,"List(false, 1738829236392, true, part-00001-240819c0-9c71-46c2-8601-b57b4abfbf6b-c000.snappy.parquet, 16604637, {""numRecords"":1341441}, List(1738828509000001, 1738828509000001, 1738828509000001, 268435456))"
null,null,"List(false, 1738829236392, true, part-00002-e0b48976-9650-4467-8879-b2add0e7aff0-c000.snappy.parquet, 16365219, {""numRecords"":1341457}, List(1738828509000002, 1738828509000002, 1738828509000002, 268435456))"
null,null,"List(false, 1738829236392, true, part-00003-4557258a-5982-4c45-8f63-5b14c1f84689-c000.snappy.parquet, 16389598, {""numRecords"":1329597}, List(1738828509000003, 1738828509000003, 1738828509000003, 268435456))"
null,null,"List(false, 1738829236392, true, part-00004-b6cd5e3b-10ac-410c-8cc9-91725fb1cd97-c000.snappy.parquet, 4108562, {""numRecords"":329209}, List(1738828509000004, 1738828509000004, 1738828509000004, 268435456))"
null,null,"List(false, 1738829236392, true, part-00005-26912275-dbf0-46f1-837e-cbdc5813d40a-c000.snappy.parquet, 16218924, {""numRecords"":1337521}, List(1738828509000005, 1738828509000005, 1738828509000005, 268435456))"
null,null,"List(false, 1738829236392, true, part-00006-2f22e9e8-4e54-44ae-af83-c64fcb2072d5-c000.snappy.parquet, 16100740, {""numRecords"":1336204}, List(1738828509000006, 1738828509000006, 1738828509000006, 268435456))"
null,null,"List(false, 1738829236392, true, part-00007-76b4a7f8-88ac-4431-b7e0-7128d2b77b70-c000.snappy.parquet, 16155979, {""numRecords"":1335997}, List(1738828509000007, 1738828509000007, 1738828509000007, 268435456))"
null,null,"List(false, 1738829236392, true, part-00008-ae4bd135-ba0c-4c52-85bc-024879eac2ee-c000.snappy.parquet, 16047557, {""numRecords"":1333645}, List(1738828509000008, 1738828509000008, 1738828509000008, 268435456))"


## Partition +  Z Order 

In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/flight_data_year_z",True)

True

In [0]:
%sql
DROP TABLE IF EXISTS flight_data_year_z;

CREATE TABLE flight_data_year_z(
   Year INT,Month INT,DayofMonth INT,DayofWeek INT,DepTime STRING, CRSDepTime INT,ArrTime STRING,CRSArrTime INT,UniqueCarrier STRING, FlightNum INT, TailNum STRING, ActualElapsedTime STRING,CRSElapsedTime STRING,AirTime STRING,
  ArrDelay STRING, DepDelay STRING, Origin STRING,Dest STRING,Distance INT, TaxiIn STRING,TaxiOut STRING,Cancelled INT, CancellationCode STRING, Diverted INT,CarrierDelay STRING, WeatherDelay STRING, NASDelay STRING, SecurityDelay STRING,LateAircraftDelay STRING)
  PARTITIONED BY (Year);

In [0]:
%python
spark.read.csv("dbfs:/databricks-datasets/asa/airlines/2*.csv",header=True,inferSchema=True)\
    .write.mode("append").saveAsTable("flight_data_year_z")

In [0]:
%fs ls dbfs:/user/hive/warehouse/flight_data_year_z/Year=2008/

path,name,size,modificationTime
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2008/part-00033-923e9541-9561-4855-a2c2-ee728bea064e.c000.snappy.parquet,part-00033-923e9541-9561-4855-a2c2-ee728bea064e.c000.snappy.parquet,26991104,1738831098000
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2008/part-00034-577066c6-3777-4b40-99a7-0c6a3456f085.c000.snappy.parquet,part-00034-577066c6-3777-4b40-99a7-0c6a3456f085.c000.snappy.parquet,26843980,1738831109000
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2008/part-00035-fbb8798f-334a-4b17-9e43-f4bd8977ef45.c000.snappy.parquet,part-00035-fbb8798f-334a-4b17-9e43-f4bd8977ef45.c000.snappy.parquet,26748290,1738831107000
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2008/part-00036-cc46eb1f-5d1e-4665-aa68-ad8ffd657813.c000.snappy.parquet,part-00036-cc46eb1f-5d1e-4665-aa68-ad8ffd657813.c000.snappy.parquet,26157729,1738831115000
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2008/part-00037-95f713d5-c0b4-4fd2-98e8-7fd1871bae26.c000.snappy.parquet,part-00037-95f713d5-c0b4-4fd2-98e8-7fd1871bae26.c000.snappy.parquet,25838570,1738831125000
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2008/part-00043-e437626a-a3ae-42c4-bbc7-54a26f62812b.c000.snappy.parquet,part-00043-e437626a-a3ae-42c4-bbc7-54a26f62812b.c000.snappy.parquet,3149821,1738831156000


we can do the z order for each partition, for each partition we can have different pattern 

In [0]:
%sql
OPTIMIZE flight_data_year_z
WHERE Year=2008
ZORDER BY (Origin);

path,metrics
dbfs:/user/hive/warehouse/flight_data_year_z,"List(1, 6, List(138973647, 138973647, 1.38973647E8, 1, 138973647), List(3149821, 26991104, 2.2621582333333332E7, 6, 135729494), 1, List(minCubeSize(107374182400), List(0, 0), List(6, 135729494), 0, List(6, 135729494), 1, null), null, 0, 1, 6, 0, false, 0, 0, 1738831180567, 1738831285266, 8, 1, null, List(0, 0), 29, 29, 83165, 0, null)"


In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled=false;
VACUUM flight_data_year_z RETAIN 0 HOURS;

path
dbfs:/user/hive/warehouse/flight_data_year_z


In [0]:
%fs ls dbfs:/user/hive/warehouse/flight_data_year_z/Year=2008/

path,name,size,modificationTime
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2008/part-00000-384a5c1e-28ab-41e3-b9bd-9947627fde42.c000.snappy.parquet,part-00000-384a5c1e-28ab-41e3-b9bd-9947627fde42.c000.snappy.parquet,138973647,1738831280000


In [0]:
%fs ls dbfs:/user/hive/warehouse/flight_data_year_z/Year=2007/

path,name,size,modificationTime
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2007/part-00028-b73e88f4-3b44-42cc-abd6-95bc4b450e7b.c000.snappy.parquet,part-00028-b73e88f4-3b44-42cc-abd6-95bc4b450e7b.c000.snappy.parquet,26983653,1738830952000
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2007/part-00029-639e4f56-182f-49cc-b310-7d5611b373cd.c000.snappy.parquet,part-00029-639e4f56-182f-49cc-b310-7d5611b373cd.c000.snappy.parquet,26719088,1738830974000
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2007/part-00030-30598078-d5bf-4955-b77c-2aad21c1b60f.c000.snappy.parquet,part-00030-30598078-d5bf-4955-b77c-2aad21c1b60f.c000.snappy.parquet,27034499,1738830968000
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2007/part-00031-7f62d306-f4c7-405a-97fe-4dc998d1c496.c000.snappy.parquet,part-00031-7f62d306-f4c7-405a-97fe-4dc998d1c496.c000.snappy.parquet,26743718,1738830970000
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2007/part-00032-f1e24736-888f-49a9-beea-4844180a1ab0.c000.snappy.parquet,part-00032-f1e24736-888f-49a9-beea-4844180a1ab0.c000.snappy.parquet,26056520,1738831101000
dbfs:/user/hive/warehouse/flight_data_year_z/Year=2007/part-00043-1ac1d537-0edc-40a6-9f1f-a796af657365.c000.snappy.parquet,part-00043-1ac1d537-0edc-40a6-9f1f-a796af657365.c000.snappy.parquet,5737131,1738831150000


we can see different files for different years , many files in 2007 where we are not doing any z order and 1 file in 2008 

now we can do z order for all the years

In [0]:
%sql
OPTIMIZE flight_data_year_z
WHERE Year>=2000 AND Year<=2007
ZORDER BY (Origin);

path,metrics
dbfs:/user/hive/warehouse/flight_data_year_z,"List(8, 41, List(73244457, 142638158, 1.0952735675E8, 8, 876218854), List(213528, 27034499, 2.0783933658536587E7, 41, 852141280), 8, List(minCubeSize(107374182400), List(0, 0), List(41, 852141280), 0, List(41, 852141280), 8, null), null, 0, 1, 41, 0, false, 0, 0, 1738831319571, 1738831928671, 8, 8, null, List(0, 0), 29, 29, 3500575, 0, null)"


now we can see we have only one file/order

#Key Take Aways from Z-order
- Z order usually more efficient than partitioning as avoid "over partition" and helps skip data more efficiently
- More columns we use for Z order , it becomes more beneficial
- Typically used for high cardinality columns
- Columns should be in the list of first 32
- Optimization does not happen automatically
- As data change , we need to re-run OPTIMIZE with Z Order
- We can Z order inside partitions
